# dev003

In [1]:
import os
import sys
import itertools
import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

import line_notify

In [2]:
from scipy import sparse
from sklearn.decomposition import NMF

In [3]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [4]:
def apk(y_true, y_pred, K=12):
    assert(len(y_true) == len(y_pred))
    apks = []
    for idx in range(len(y_true)):
        y_i_true = y_true[idx]
        y_i_pred = y_pred[idx]

        # 予測値の数と重複の確認
        assert(len(y_i_pred) <= K)
        assert(len(np.unique(y_i_pred)) == len(y_i_pred))

        sum_precision = 0.0
        num_hits = 0.0

        for i, p in enumerate(y_i_pred):
            if p in y_i_true:
                num_hits += 1
                precision = num_hits / (i+1)
                sum_precision += precision
        apk = sum_precision / min(len(y_i_true), K)
        apks.append(apk)
    return apks

In [5]:
SEED = 42

ディレクトリ設定

In [6]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]
exp_name = 'dev003'
#os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

データ読み込み

In [7]:
articles = pd.read_csv(INPUT_DIR + 'articles.csv', dtype='object')
customers = pd.read_csv(INPUT_DIR + 'customers.csv')
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv', dtype={'article_id':'str'}, parse_dates=['t_dat'])
sample = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

In [12]:
transactions['article_id'].nunique()

104547

In [15]:
history = transactions[transactions['t_dat'] > '2020-09-16'].copy()

In [13]:
from tqdm import tqdm

In [18]:
ITEM_IDS = history['article_id'].unique().tolist()

In [19]:
len(ITEM_IDS)

17265

In [34]:
pairs = {}
for item in tqdm(ITEM_IDS):
    users = history.loc[history['article_id']==item, 'customer_id'].unique().tolist()
    vc = history.loc[(history['customer_id'].isin(users))&(history['article_id']!=item), 'article_id'].value_counts()
    if len(vc) > 0:
        pairs[item] = vc.index[0]


100%|██████████| 17265/17265 [06:24<00:00, 44.85it/s]


In [47]:
df = transactions.head(1000)

In [50]:
df['article_id'] = df['article_id'].map(pairs)

/tmp/ipykernel_18298/184521016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article_id'] = df['article_id'].map(pairs)


In [52]:
df[['customer_id', 'article_id']].dropna().drop_duplicates()

,customer_id,article_id
69,00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...,0845046004
72,00402f4463c8dc1b3ee54abfdea280e96cd87320449eca...,0889456001
79,0051a1d00ec9a4ff1a2f9aa32a4bb054cc9ba08e80be59...,0889016001
86,00609a1cc562140fa87a6de432bef9c9f0b936b259ad30...,0673677022
88,00609a1cc562140fa87a6de432bef9c9f0b936b259ad30...,0885951001
...,...,...
972,0555f0db4a2694a62c7842440ca57a8e77812be9723c9b...,0610776001
974,05701e4bc2f83b8ebd680fc7984978a46d961a2eb4532e...,0821513005
976,05701e4bc2f83b8ebd680fc7984978a46d961a2eb4532e...,0915529003
979,0572ee4d4c148508a6a40bbb3349a8e6ca285097f3843e...,0850917001
